In [1]:
import numpy as np 
import pandas as pd 
import shutil as sh
import os
from tqdm.auto import tqdm

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1472 (delta 40), reused 43 (delta 17), pack-reused 1394
Receiving objects: 100% (1472/1472), 5.92 MiB | 22.62 MiB/s, done.
Resolving deltas: 100% (978/978), done.


In [3]:
!mv yolov5/* ./

In [4]:
#!pip install -r requirements.txt

In [5]:
df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [6]:
df.head(2)

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0


In [7]:
index = list(set(df.image_id))

In [8]:
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [9]:
!python train.py --img 1024 --batch 2 --epochs 40 --data ../input/configyolo5/wheat0.yaml --cfg ../input/panet-config/yolo5_PANET.yaml --name yolov5x_fold4 --weights ../input/selfyolo5/last_yolov5x_fold4.pt

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(batch_size=2, bucket='', cache_images=False, cfg='../input/panet-config/yolo5_PANET.yaml', data='../input/configyolo5/wheat0.yaml', device='', epochs=40, evolve=False, hyp='', img_size=[1024], multi_scale=False, name='yolov5x_fold4', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='../input/selfyolo5/last_yolov5x_fold4.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-07-13 12:02:30.934610: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
Hyperparameters {'optimizer': 'SGD', 'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 

In [10]:
#!python train.py --img 1024 --batch 2 --epochs x --data ../input/configyolo5/wheat0.yaml --cfg ../input/panet-config/yolo5_PANET.yaml --name yolov5x_fold4 --weights ../input/panet-config/PANet_0709_v1.pt